<a href="https://colab.research.google.com/github/kimtoto23/Network_Simulation_CMCP/blob/main/networksimulation_optimization_cmcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connected Maximum Coverage Problem using GA Algorithm

##### google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### package import

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot
import random
import matplotlib.pyplot as plt
import math
import sys
import heapq
from itertools import chain
from collections import defaultdict
import time
import itertools
from PIL import Image
import copy
import random
import pickle

#### Dataset: Node & Edge weight

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UAM_입지선정/유전알고리즘/edge.csv', index_col= 0)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,15,0,0,24,0,18,0,0,0,0,0
1,15,0,22,0,0,0,0,0,0,0,0,0
2,0,22,0,18,16,0,0,0,20,0,0,0
3,0,0,18,0,0,12,0,0,0,0,0,0
4,24,0,16,0,0,0,25,12,24,0,0,0
5,0,0,0,12,0,0,0,0,12,0,0,22
6,18,0,0,0,25,0,0,15,0,22,0,0
7,0,0,0,0,12,0,15,0,30,0,25,0
8,0,0,20,0,24,12,0,30,0,0,19,19
9,0,0,0,0,0,0,22,0,0,0,19,0


In [ ]:
node_edge = pd.DataFrame({"node": [0,1,2,3,4,5,6,7,8,9,10,11], "demand":[15,10,12,18,5,24,11,16,13,22,19,20]})
node_edge

,node,demand
0,0,15
1,1,10
2,2,12
3,3,18
4,4,5
5,5,24
6,6,11
7,7,16
8,8,13
9,9,22


# Reconstruction  of Graph using Dijkstra
---

In [ ]:
def dijkstra_map(df):
  df.columns= df.columns.astype('str')
  df = df.T
  df.columns= df.columns.astype('str')
  df = df.T
  inf = sys.maxsize
  df = df.replace(0,inf)
  df_dict = df.to_dict()

  re_df = pd.DataFrame()
  for start in df.columns:
    distances = {node: float('inf') for node in df_dict}  # start로 부터의 거리 값을 저장하기 위함
    distances[start] = 0  # 시작 값은 0이어야 함
    queue = []
    heapq.heappush(queue, [distances[start], start])  # 시작 노드부터 탐색 시작 하기 위함.

    while queue:  # queue에 남아 있는 노드가 없으면 끝
      current_distance, current_destination = heapq.heappop(queue)  # 탐색 할 노드, 거리를 가져옴.

      if distances[current_destination] < current_distance:  # 기존에 있는 거리보다 길다면, 볼 필요도 없음
        continue

      for new_destination, new_distance in df_dict[current_destination].items():
        distance = current_distance + new_distance  # 해당 노드를 거쳐 갈 때 거리
        if distance < distances[new_destination]:  # 알고 있는 거리 보다 작으면 갱신
          distances[new_destination] = distance
          heapq.heappush(queue, [distance, new_destination])  # 다음 인접 거리를 계산 하기 위해 큐에 삽입

    re_df = pd.concat([re_df, pd.DataFrame.from_dict(distances, orient = 'index')], axis = 1)

  re_df.columns = df.columns.astype('int')
  re_df = re_df.T

  return re_df

# Optimization CMCP

In [ ]:
# !pip install pulp

In [ ]:
# from pulp import *

In [ ]:
# prob = LpProblem("The_FLP",LpMaximize)  #("아무거나이름",최소최대,,,)

In [ ]:
! pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.3 which is incompatible.


In [ ]:
from ortools.linear_solver import pywraplp
import ortools

In [ ]:
# from ortools.init import pywrapinit

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UAM_입지선정/유전알고리즘/edge.csv', index_col= 0)
dat = dijkstra_map(df)

In [ ]:
def find_coverage_node(graph, length, node):   ### graph: 데이터프레임, length: coverage length, node: chromosome node  eg)  [[2, 6, 7], [7, 9, 10], [2, 3, 4], [2, 7, 10], [2, 5, 7]]     => return chromosome

  inf = sys.maxsize

  graph = graph.replace(0,inf)
  re = []
  coverage = copy.deepcopy(node)
  i = 0
  for i in range(len(coverage)):
    cov_node = []
    for j in coverage[i]:
      cov_node.append(graph[j].loc[graph[j] <= length ].index.tolist())

    coverage[i].extend(sum(cov_node,[]))
    coverage[i] = list(set(coverage[i]))

  return coverage

In [ ]:
def objective_function(graph_weight, node):
  total_demand = []
  i = 0
  for i in range(len(node)):
    total_demand.append(int(sum((node_edge.loc[node_edge.node.isin(node[i]), ['demand']]).values)))

  return total_demand

In [ ]:
## Problem Conditions
ax0_plant = 12
ax1_plant = 12
P = 3

cost_matrix = dat.to_numpy().tolist()

ax0_demand = [15, 10, 12, 18, 5, 24, 11, 16, 13, 22, 19, 20]
ax1_demand = [15, 10, 12, 18, 5, 24, 11, 16, 13, 22, 19, 20]

## Define Solver
solver = pywraplp.Solver.CreateSolver('SCIP')

## Define Decision Variables
x = {}
z = {}
w = {}

for i in range(ax0_plant):
    for j in range(ax1_plant):
        x[i, j] = solver.NumVar(0, 1, '')  # Change variable range to binary (0 or 1)

# Define z variable for coverage by facility
for i in range(ax0_plant):
    z[i] = solver.BoolVar('z[%i]' % i)

# Define w variables for connectivity
for n in range(ax0_plant):
    for j in range(ax1_plant):
        w[n, j] = solver.BoolVar('w[%i,%i]' % (n, j))

for j in range(ax1_plant):
    for l in range(ax1_plant):
        w[j, l] = solver.BoolVar('w[%i,%i]' % (j, l))

## Define Constraints

## Coverage Constraints
for i in range(ax0_plant):
    for j in range(ax1_plant):
        # Add constraints to link x and z variables
        solver.Add(x[i, j] <= z[i] * 30)  # Assuming 30 is the coverage distance

## Connectivity Constraints
for n in range(ax0_plant):
    for j in range(ax1_plant):
        solver.Add(x[n, j] <= solver.Sum(w[n, l] for l in range(ax1_plant)) * 20)  # Assuming 20 is the connectivity distance

for j in range(ax1_plant):
    for l in range(ax1_plant):
        solver.Add(w[j, l] == w[l, j])  # Ensure symmetry of connectivity

## Count Constraint: Summation of x[i,j] = P for each facility, P is new facility count
for i in range(ax0_plant):
    solver.Add(solver.Sum(x[i, j] for j in range(ax1_plant)) == P)  # Change the value to the desired maximum count

## Define Objective Function
objective_terms = []

for i in range(ax0_plant):
    # Modify the objective function to maximize coverage
    objective_terms.append(ax0_demand[i] * z[i])

solver.Maximize(solver.Sum(objective_terms))

## Invoke the solver
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in range(ax0_plant):
        for j in range(ax1_plant):
            print('x({},{}) ='.format(i, j), round(x[i, j].solution_value()))
    print('Facility coverage:')
    for i in range(ax0_plant):
        print('z({}) ='.format(i), round(z[i].solution_value()))

else:
    print('The problem does not have an optimal solution.')

Objective value = 185.0
x(0,0) = 1
x(0,1) = 1
x(0,2) = 0
x(0,3) = 0
x(0,4) = 0
x(0,5) = 1
x(0,6) = 0
x(0,7) = 0
x(0,8) = 0
x(0,9) = 0
x(0,10) = 0
x(0,11) = 0
x(1,0) = 1
x(1,1) = 0
x(1,2) = 0
x(1,3) = 0
x(1,4) = 0
x(1,5) = 0
x(1,6) = 0
x(1,7) = 0
x(1,8) = 1
x(1,9) = 0
x(1,10) = 0
x(1,11) = 1
x(2,0) = 1
x(2,1) = 1
x(2,2) = 0
x(2,3) = 0
x(2,4) = 0
x(2,5) = 0
x(2,6) = 0
x(2,7) = 0
x(2,8) = 0
x(2,9) = 1
x(2,10) = 0
x(2,11) = 0
x(3,0) = 1
x(3,1) = 0
x(3,2) = 1
x(3,3) = 0
x(3,4) = 0
x(3,5) = 0
x(3,6) = 0
x(3,7) = 0
x(3,8) = 0
x(3,9) = 0
x(3,10) = 1
x(3,11) = 0
x(4,0) = 1
x(4,1) = 0
x(4,2) = 0
x(4,3) = 0
x(4,4) = 0
x(4,5) = 1
x(4,6) = 0
x(4,7) = 1
x(4,8) = 0
x(4,9) = 0
x(4,10) = 0
x(4,11) = 0
x(5,0) = 1
x(5,1) = 0
x(5,2) = 0
x(5,3) = 0
x(5,4) = 1
x(5,5) = 0
x(5,6) = 0
x(5,7) = 0
x(5,8) = 1
x(5,9) = 0
x(5,10) = 0
x(5,11) = 0
x(6,0) = 1
x(6,1) = 0
x(6,2) = 0
x(6,3) = 0
x(6,4) = 0
x(6,5) = 0
x(6,6) = 0
x(6,7) = 1
x(6,8) = 1
x(6,9) = 0
x(6,10) = 0
x(6,11) = 0
x(7,0) = 1
x(7,1) = 0
x(7,2) = 1
x(7,3